# Q1

In [2]:
import numpy as np
from scipy.stats import nbinom, binom, uniform
import matplotlib.pyplot as plt
import scipy.linalg as la
r = 2  
q = 0.4
p = 0.1  
n_states = 11
sanity_print_checks = False
state_space = range(n_states)


P = np.zeros((len(state_space), len(state_space)))
ps_patients_arrive = [nbinom.pmf(j, r, q) for j in range(n_states-1)]
ps_patients_arrive.append(1 - sum(ps_patients_arrive)) # Remembering that more than 10 patients could arrive, but since we could never admit them anyway, we will just add them to the last possibility

for i in state_space:
    num_patients = i
    ps_patients_healthy = [binom.pmf(j, num_patients, 1-p) for j in range(num_patients+1)]

    # We can no loop over all possible transitions
    tranistions = len(ps_patients_healthy) * len(ps_patients_arrive)
    for k in range(tranistions):
        new_healthy_patients = k % len(ps_patients_healthy)
        new_arrive_patients = k // len(ps_patients_healthy)

        if num_patients - new_healthy_patients + new_arrive_patients <= 10:
            j = num_patients - new_healthy_patients + new_arrive_patients
            P[i,j] += ps_patients_healthy[new_healthy_patients] * ps_patients_arrive[new_arrive_patients]

            if uniform.rvs() <= 0.02 and  sanity_print_checks:
                print(f"Viable transition when starting in {num_patients} and adding {new_arrive_patients} and removing {new_healthy_patients}")
        else:
            # If we would exceed the maximum number of patients, we will instead transition to the maximum number of patients
            j = n_states - 1
            P[i,j] += ps_patients_healthy[new_healthy_patients] * ps_patients_arrive[new_arrive_patients]

            if uniform.rvs() <= 0.02 and sanity_print_checks:
                print(f"Declining transition when starting in {num_patients} and adding {new_arrive_patients} and removing {new_healthy_patients}")

print(P)
# Sanity check that the rows should sum to 1
print(np.sum(P, axis=1))

[[0.16       0.192      0.1728     0.13824    0.10368    0.0746496
  0.05225472 0.03583181 0.02418647 0.01612431 0.03023309]
 [0.144      0.1888     0.17472    0.141696   0.107136   0.07755264
  0.05449421 0.0374741  0.025351   0.01693053 0.03184552]
 [0.1296     0.18432    0.176128   0.1449984  0.110592   0.08051098
  0.05680005 0.03917611 0.02656331 0.01777258 0.03353857]
 [0.11664    0.178848   0.1769472  0.14811136 0.11403264 0.08351908
  0.05917114 0.0409385  0.02782459 0.01865165 0.03531583]
 [0.104976   0.1726272  0.17713728 0.15099494 0.11744051 0.08657043
  0.06160594 0.04276177 0.02913598 0.01956894 0.037181  ]
 [0.0944784  0.16586208 0.17668627 0.15360918 0.12079596 0.08965744
  0.06410239 0.04464619 0.03049856 0.02052565 0.03913789]
 [0.08503056 0.15872371 0.17560385 0.15591689 0.12407728 0.09277129
  0.06665789 0.04659181 0.03191332 0.02152294 0.04119045]
 [0.0765275  0.1513544  0.17391584 0.15788558 0.12726124 0.09590189
  0.06926923 0.04859841 0.03338117 0.02256198 0.043

# Q2

In [ ]:
X0 = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

X12 = np.dot(X0, np.linalg.matrix_power(P, 12))
print(X12)
mean_number_of_patients_12 = np.dot(X12, state_space)
print(mean_number_of_patients_12)

# Q3

In [3]:
w, vl, vr = la.eig(a = P, left = True)
# Check eigenvalue is 1
print(w[0])
pi = vl[:,0]
pi = pi / np.sum(pi)

# Check that pi is a stationary distribution
print(np.allclose(np.dot(pi, P), pi))

mean_number_of_patients = np.dot(pi, state_space)
print(mean_number_of_patients)

# Variance of expected number of patients
state_space = np.array(state_space)
variance = np.dot(pi, state_space ** 2) - mean_number_of_patients ** 2
print(np.dot(pi, state_space ** 2))
print(variance)

(0.9999999999999991+0j)
True
3.260544862314694
17.316634813008026
6.685482013841277


### Q3 - Simulation
Just to check the theoretical values are correct

In [4]:
n = 100000
es = np.zeros(n)
for i in range(n):
    # Select a random state from the stationary distribution
    state = np.random.choice(state_space, p=pi)
    es[i] = state

print(np.mean(es))
print(np.var(es))

3.25232
6.6422546176


# Q4

In [7]:
P_abs = P.copy()
P_abs[0,1:11] = 0
P_abs[10,0:10] = 0
P_abs[10,10] = 1
P_abs[0,0] = 1

x0 = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0,0])
months = 24
x24 = np.dot(x0, np.linalg.matrix_power(P_abs, months))
print(x24)
print(x24[0] + x24[10])

[7.35933720e-01 4.20383111e-03 4.22164919e-03 3.57472091e-03
 2.77214749e-03 2.03985244e-03 1.44985953e-03 1.00548492e-03
 6.84632478e-04 4.59584404e-04 2.43654518e-01]
0.9795882375313745


### Q4 - Simulation
To verify previous code block

In [6]:
n = 10000
end_states = np.zeros(n)
month = 0
max_months = 24
converged = False
for i in range(n):
    state = 5 
    while not converged and month < max_months:
        state = np.random.choice(state_space, p=P_abs[state, :])
        if state == 0 or state == 10:
            end_states[i] = state
            converged = True
        month += 1
    if not converged:
        end_states[i] = state
    month = 0
    converged = False
print((sum(end_states == 10) + sum(end_states == 0)) / len(end_states))

0.9783


# Q5

In [9]:
P_abs_mod = P_abs.copy()

# Move collumn
P_abs_mod = np.insert(P_abs_mod, 10, P_abs_mod[:, 0], axis=1)
P_abs_mod = np.delete(P_abs_mod, 0, axis=1)
# Move row
P_abs_mod = np.insert(P_abs_mod, 10, P_abs_mod[0, :], axis=0)
P_abs_mod = np.delete(P_abs_mod, 0, axis=0)

R = P_abs_mod[:9, -2:]
Q = P_abs_mod[:9, :9]
print(R)

IQ_inv = np.linalg.inv(np.eye(9) - Q)
U = np.dot(IQ_inv, R)
print(U)

print("Probability of absorption into state 10 given x0 = 5")
print(U[5, 1])

[[0.144      0.03184552]
 [0.1296     0.03353857]
 [0.11664    0.03531583]
 [0.104976   0.037181  ]
 [0.0944784  0.03913789]
 [0.08503056 0.04119045]
 [0.0765275  0.04334275]
 [0.06887475 0.04559895]
 [0.06198728 0.04796334]]
[[0.76965139 0.23034861]
 [0.76472962 0.23527038]
 [0.76007301 0.23992699]
 [0.75564643 0.24435357]
 [0.75141791 0.24858209]
 [0.7473583  0.2526417 ]
 [0.74344101 0.25655899]
 [0.73964171 0.26035829]
 [0.73593813 0.26406187]]
Probability of absorption into state 10 given x0 = 5
0.2526416982841323


# Q6

In [12]:
M = 200
n = 5000
ps = np.zeros(M)
for m in range(M):
    max_months = 24
    succeses = []
    for i in range(n):
        state = 5 
        month = 0
        while True:
            month += 1
            state = np.random.choice(state_space, p=P_abs[state, :])
            if state == 0:
                break
            if state == 10:
                succeses.append(month)
                break
            
    succeses = np.array(succeses)
    ps[m] = np.sum(succeses <= 24)/ len(succeses)
print(np.mean(ps))
print(f"CI: {np.percentile(ps, 2.5)} - {np.percentile(ps, 97.5)}")


0.9801099133848956
CI: 0.9703881203007519 - 0.9881679676629058
